In [50]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [51]:
pff = pd.read_csv('pff.csv', index_col=0)
pff.head(2)

,Team,Opposing Team,Season,Week,Date,Time,H/A,W/L,PF,PA,OVR,OFF,PASS,PBLK,RECV,RUN,RBLK,DEF,RDEF,TACK,PRSH,COV,SPEC,OVR_opp,OFF_opp,PASS_opp,PBLK_opp,RECV_opp,RUN_opp,RBLK_opp,DEF_opp,RDEF_opp,TACK_opp,PRSH_opp,COV_opp,SPEC_opp
0,Arizona Cardinals,San Francisco 49Ers,2006,1,09/10/2006,4:15pm,H,W,34,27,72.5,72.6,75.5,72.4,76.5,47.4,65.6,67.7,69.8,37.5,56.2,70.7,-,63.8,64.7,54.5,83.6,62.0,67.4,63.6,61.4,63.2,32.0,69.0,53.8,-
1,Arizona Cardinals,Seattle Seahawks,2006,2,09/17/2006,4:05pm,A,L,10,21,48.0,50.1,27.9,57.1,65.3,59.7,62.1,49.8,38.3,44.7,60.5,62.3,-,80.2,70.2,56.5,68.6,57.6,63.5,85.4,85.5,72.8,61.2,73.4,84.0,-


In [52]:
weeks = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17']
pff = pff[pff['Week'].isin(weeks)]

In [53]:
pff['Week'] = pff['Week'].astype(int)
pff['Date'].map(lambda x : x[:2] < '05').value_counts()

# shows that there are games in january during reg season
# viewing dataframe you can see that the dates are a year off when its in january

False    7008
True      160
Name: Date, dtype: int64

In [54]:
# fixes year-off dates in January and converts date to the correct format

def fix_date(s):
    date = datetime.datetime.strptime(s, '%m/%d/%Y')
    if (date.month < 5):
        date = date.replace(year = date.year + 1)
    return date.isoformat()[:10]

pff['Date'] = pff['Date'].map(fix_date)

In [55]:
# check that dates in January have the correct year/season

pff[pff['Date'].map(lambda x : x[5:7] < '05')]

,Team,Opposing Team,Season,Week,Date,Time,H/A,W/L,PF,PA,OVR,OFF,PASS,PBLK,RECV,RUN,RBLK,DEF,RDEF,TACK,PRSH,COV,SPEC,OVR_opp,OFF_opp,PASS_opp,PBLK_opp,RECV_opp,RUN_opp,RBLK_opp,DEF_opp,RDEF_opp,TACK_opp,PRSH_opp,COV_opp,SPEC_opp
67,Arizona Cardinals,Green Bay Packers,2009,17,2010-01-03,4:15pm,H,L,7,33,55.4,54.3,37.3,62.1,63.2,56.4,59.2,58.3,64.7,72.8,52.2,55.8,-,79.6,69.3,74.5,89.3,69.6,39.2,69.9,88.8,74.1,68.7,64.3,90.2,-
85,Arizona Cardinals,San Francisco 49Ers,2010,17,2011-01-02,4:15pm,A,L,7,38,56.3,54.6,45.8,75.6,51.2,72.5,58.9,61.3,60.3,74.9,59.3,62.0,-,85.6,79.3,68.6,80.1,73.3,63.5,79.1,83.7,73.7,81.9,62.0,88.9,-
101,Arizona Cardinals,Seattle Seahawks,2011,17,2012-01-01,4:15pm,H,W,23,20,65.3,67.9,51.6,74.3,70.5,61.6,72.6,59.6,44.0,65.0,59.3,74.4,-,73.4,75.1,66.6,79.1,64.4,71.0,83.4,66.0,60.2,55.6,62.6,70.3,-
166,Arizona Cardinals,Seattle Seahawks,2015,17,2016-01-03,4:25pm,H,L,6,36,58.4,56.4,68.9,56.4,53.7,54.6,50.9,68.1,69.4,66.1,59.9,66.1,44.7,84.1,73.3,63.7,67.3,73.9,71.6,64.2,86.7,91.6,86.0,71.5,78.5,64.9
184,Arizona Cardinals,Los Angeles Rams,2016,17,2017-01-01,4:25pm,A,W,44,6,85.2,69.0,62.4,65.3,72.1,53.1,65.4,91.4,79.7,57.6,81.0,91.7,47.3,54.3,45.0,35.2,51.1,50.1,58.4,52.8,65.7,65.3,67.3,74.9,56.6,70.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7313,Washington Redskins,Los Angeles Chargers,2009,17,2010-01-03,4:15pm,A,L,20,23,60.9,61.2,57.6,67.6,62.6,43.7,69.7,60.1,67.5,54.6,60.4,54.0,-,69.6,65.4,53.8,71.8,70.2,53.2,65.6,71.3,72.1,64.8,65.4,67.5,-
7329,Washington Redskins,New York Giants,2010,17,2011-01-02,4:15pm,H,L,14,17,74.7,74.2,80.5,56.2,72.8,50.8,73.7,69.1,83.6,85.7,59.9,53.2,-,69.0,72.9,89.3,74.3,69.0,49.0,61.6,61.4,68.7,73.6,85.0,43.1,-
7345,Washington Redskins,Philadelphia Eagles,2011,17,2012-01-01,1:00pm,A,L,10,34,65.8,65.9,58.4,43.0,59.9,68.5,85.1,63.3,75.2,59.5,65.9,51.5,-,80.3,84.8,79.6,72.2,84.4,63.7,67.4,67.3,46.5,65.3,77.3,74.3,-
7410,Washington Redskins,Dallas Cowboys,2015,17,2016-01-03,1:00pm,A,W,34,23,66.1,72.4,64.1,74.7,66.2,54.9,79.3,52.5,50.1,70.0,54.6,55.1,68.5,75.8,82.4,76.1,89.7,76.8,33.8,89.1,57.4,64.1,72.3,60.5,46.4,58.4


In [56]:
# create pff home and pff away and sort so that way games line up
pff_home = pff[pff['H/A']=='H'].sort_values(by=['Season','Week','Date','Time','Team']).reset_index().reset_index().drop(columns='index').rename(columns={'level_0':'gamenum'})
pff_away = pff[pff['H/A']=='A'].sort_values(by=['Season','Week','Date','Time','Opposing Team']).reset_index().reset_index().drop(columns='index').rename(columns={'level_0':'gamenum'})

# check
test_match = pd.concat([pff_home[['gamenum', 'Team', 'Opposing Team', 'Season', 'Week', 'Date', 'Time']],
          pff_away[['gamenum', 'Team', 'Opposing Team', 'Season', 'Week', 'Date', 'Time']]],
         axis=1, ignore_index=True)

test_match.columns = ['gamenum1','Home_Team1','Away_Team1','Season1','Week1','Date1','Time1',
                  'gamenum2','Away_Team2','Home_Team2','Season2','Week2','Date2','Time2']

# this should be blank
test_match[test_match['Home_Team1'] != test_match['Home_Team2']]

,gamenum1,Home_Team1,Away_Team1,Season1,Week1,Date1,Time1,gamenum2,Away_Team2,Home_Team2,Season2,Week2,Date2,Time2


In [57]:
pff_fixed = pd.concat([pff_home,pff_away], axis=0).sort_values(by=['gamenum','H/A'])
pff_fixed

,gamenum,Team,Opposing Team,Season,Week,Date,Time,H/A,W/L,PF,PA,OVR,OFF,PASS,PBLK,RECV,RUN,RBLK,DEF,RDEF,TACK,PRSH,COV,SPEC,OVR_opp,OFF_opp,PASS_opp,PBLK_opp,RECV_opp,RUN_opp,RBLK_opp,DEF_opp,RDEF_opp,TACK_opp,PRSH_opp,COV_opp,SPEC_opp
0,0,Miami Dolphins,Pittsburgh Steelers,2006,1,2006-09-07,8:30pm,A,L,17,28,65.5,64.7,50.3,65.6,69.1,57.0,65.8,64.4,69.9,67.8,80.2,44.8,-,74.2,68.3,65.5,45.2,73.5,60.6,64.1,77.0,68.6,59.9,73.2,72.0,-
0,0,Pittsburgh Steelers,Miami Dolphins,2006,1,2006-09-07,8:30pm,H,W,28,17,74.2,68.3,65.5,45.2,73.5,60.6,64.1,77.0,68.6,59.9,73.2,72.0,-,65.5,64.7,50.3,65.6,69.1,57.0,65.8,64.4,69.9,67.8,80.2,44.8,-
1,1,Atlanta Falcons,Carolina Panthers,2006,1,2006-09-10,1:00pm,A,W,20,6,78.9,65.6,47.5,63.1,63.9,61.1,73.0,89.8,67.4,78.9,76.3,89.9,-,60.7,60.5,40.3,74.0,64.3,62.1,72.0,60.7,68.8,68.6,66.0,51.6,-
1,1,Carolina Panthers,Atlanta Falcons,2006,1,2006-09-10,1:00pm,H,L,6,20,60.7,60.5,40.3,74.0,64.3,62.1,72.0,60.7,68.8,68.6,66.0,51.6,-,78.9,65.6,47.5,63.1,63.9,61.1,73.0,89.8,67.4,78.9,76.3,89.9,-
2,2,New Orleans Saints,Cleveland Browns,2006,1,2006-09-10,1:00pm,A,W,19,14,68.4,66.9,58.6,86.2,72.0,50.4,66.0,67.4,81.6,61.8,59.4,57.8,-,66.3,60.9,57.2,68.7,60.7,58.8,57.0,70.5,71.8,71.9,53.2,68.0,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3581,3581,Los Angeles Rams,Arizona Cardinals,2019,17,2019-12-29,4:25pm,H,W,31,24,70.8,72.9,67.5,75.5,76.9,63.9,54.0,61.5,86.1,71.9,60.6,53.3,64.8,66.0,67.9,69.0,76.2,67.8,60.9,48.2,59.6,69.8,61.6,59.6,51.6,62.8
3582,3582,Philadelphia Eagles,New York Giants,2019,17,2019-12-29,4:25pm,A,W,34,17,80.5,71.5,68.3,73.4,69.5,61.5,65.2,78.8,75.8,82.1,69.4,80.4,69.6,60.8,60.6,58.3,64.4,61.8,62.3,48.6,61.4,64.1,55.9,57.7,59.5,52.8
3582,3582,New York Giants,Philadelphia Eagles,2019,17,2019-12-29,4:25pm,H,L,17,34,60.8,60.6,58.3,64.4,61.8,62.3,48.6,61.4,64.1,55.9,57.7,59.5,52.8,80.5,71.5,68.3,73.4,69.5,61.5,65.2,78.8,75.8,82.1,69.4,80.4,69.6
3583,3583,San Francisco 49Ers,Seattle Seahawks,2019,17,2019-12-29,8:20pm,A,W,26,21,79.9,88.9,88.3,81.6,84.6,72.2,69.2,64.7,65.6,66.5,58.7,65.5,56.3,60.3,70.3,67.1,76.2,67.5,58.9,68.1,39.7,37.0,26.6,61.1,42.6,66.9


In [58]:
pff_fixed.to_csv('pff_pi.csv')